In [1]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook # tqdm – библиотека для визуализации прогресса в цикле
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
sns.set(font_scale=1.5, palette='Set2')

In [50]:
def quantile_criterion(sample, q, alpha):
    estim_dist = sm.nonparametric.KDEUnivariate(sample)
    estim_dist.fit() # less -- greater FPR.
    bw = estim_dist.bw

    estim_dist = sm.nonparametric.KDEUnivariate(sample)
    estim_dist.fit(bw=bw * q)
    q_estim = np.quantile(sample, q)
    f2 = estim_dist.evaluate(q_estim)[0] ** 2
    
    sigma = np.sqrt((q * (1 - q)) / (f2 * len(sample)))
    z_alpha = sps.norm().ppf(1 - alpha/2)
    left = q_estim - z_alpha * sigma
    right = q_estim + z_alpha * sigma
    return {
        "estimated q": q_estim,
        "left bound": left,
        "right bound": right,
        "CI length": right - left
    }

sample = sample_dist.rvs(10000)
results = quantile_criterion(sample, q, 0.05)

In [57]:
np.random.seed(8)
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 1000
q = 0.5
sample_dist = sps.pareto(b=2, scale=1000)
for i in tqdm_notebook(range(N)):
    sample = sample_dist.rvs(10000)
    results = quantile_criterion(sample, q, 0.05)
    

    # Проверяю, корректен ли доверительный интервал
    if results["left bound"] > sample_dist.ppf(q) or results["right bound"] < sample_dist.ppf(q):
        bad_cnt += 1

# Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

  0%|          | 0/1000 [00:00<?, ?it/s]

Реальный уровень значимости: 0.058; доверительный интервал: [0.0451, 0.0742]


In [5]:
np.random.seed(8)
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 1000
q = 0.05
sample_dist = sps.pareto(b=2, scale=1000)
for i in tqdm_notebook(range(N)):
    sample = sample_dist.rvs(100000)
    results = quantile_criterion(sample, q, 0.05)
    

    # Проверяю, корректен ли доверительный интервал
    if results["left bound"] > sample_dist.ppf(q) or results["right bound"] < sample_dist.ppf(q):
        bad_cnt += 1

# Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

  0%|          | 0/1000 [00:00<?, ?it/s]

Реальный уровень значимости: 0.054; доверительный интервал: [0.0416, 0.0698]
